# Trading Bot for Only Long Positions

I have been developing this bot for the last 2 months alone with Udemy courses and my knowledge in fundamental analysis.


To make it work, you will need:

- An account on Binance to connect to the API
- Confgure the internal clock of the computer in UTC time (to stream the data in real-time)
- Download the following packages:
    - Pandas
    - NumPy
    - Python-Binance

We need to use our user in Binance t get the **API Keys** in order to connect ourselves to the Binance server.

This will provide us two keys that **should never be shared**, the API Key and the secret API key. In our case we should be the keys provided for the paper trading accounts so we can make experiments there.

The bot works after making some experiments and **Exploratory Data Analysis**, that is provided in another notebook that provides the bases for our strategy.

Here, we create the class **LongOnlyTrader()** as it is only using long positions.
Note: Currently developing strategies for short positions, meanwhile I provide this bot.

The bot works in the following way:

1. The variable client connect ourselves to the Binance servers so we can make trades.
2. We define the LongOnlyTrader class that has this characteristics:

    1. It uses the *symbol* to know the pairs that we will be operating, the time interval defined as *bar_length*, and the parameters for our strategy, which are *volume_thresh* and *volume_thresh*. *units* represents the quantity that we are trading (buying or selling) and *position* if it´s long or neutral (Converted to dummy variable)

    2. *start_trading()* just initializes the process calling the Web Socket and two other functions, *get_most_recent()* and *start_kline_socket()*.

    3. *get_most_recent()* returns the API information completely clean and updated, while *start_kline_socket()* is what makes the trades in real time with *stream_candles()*.

    4. The information is updated with *define_strategy()* to change the position and *execute_trades()* to buy and sell the assets.

    5. Finally, *report_trade()* returns the information of the trade that has been done in that exact moment with feedback.

3. Lastly, we define the class with the parameters that we are interested in and we just make it work with asynchronic programming!
    
4. We can have the feedback of our operations with *trader.prepared_data* and other calls such as the *trader.trade_values*, etc for *Backtesting* and *Forward Testing*.


In [21]:
from binance.client import Client
from binance import ThreadedWebsocketManager, AsyncClient, BinanceSocketManager
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from trial_api_keys import paper_api_key, paper_secret_api_key   # Change values in the trial_api_keys.py file!

In [15]:
client = Client(api_key = paper_api_key, api_secret= paper_secret_api_key, tld = "com", testnet = True)

** NOTE**: Inside stream_candles(), specify **the date that you want the bot to finish**. 

In [16]:
class LongOnlyTrader():

    def __init__(self, symbol, bar_length, return_thresh, volume_thresh, units, position = 0):

        self.symbol = symbol
        self.bar_length = bar_length
        self.data = pd.DataFrame(columns = ["Open","High","Low","Close","Volume","Complete"])
        self.available_intervals = ["1m","3m","5m","15m","30m","1h","2h","4h","6h","8h","12h","1d","3d","1w","1M"]
        self.units = units 
        self.position = position
        self.trades = 0             #NUEVO
        self.trade_values = []      #NUEVO


        #*************************** Para añadir atributos específicos según nuestra estrategia******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #**********************************************************************************

    # Añadimos los métodos get_most_recent y start_trading que se mantienen iguales
    def start_trading(self, historical_days):

        self.twm = ThreadedWebsocketManager()
        self.twm.start()

        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol=self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback=self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
    
    def get_most_recent(self, symbol,interval, days):

        now = datetime.now()
        past = str(now - timedelta(days=days))

        bars = client.get_historical_klines(symbol=symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time","Open","High","Low","Close","Volume",
                    "Close Time","Quote Asset Value","Number of Trades",
                    "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date","Open","High","Low","Close","Volume"]].copy()
        df.set_index("Date",inplace=True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for now in range(len(df)-1)] + [False]
        self.data = df

    def stream_candles(self,msg):
            
        # Extract the required items from msg    
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first = float(msg["k"]["o"])
        high = float(msg["k"]["h"])
        low = float(msg["k"]["l"])
        close = float(msg["k"]["c"])
        volume = float(msg["k"]["v"])
        complete = msg["k"]["x"]

        # STOP Trading Session IMPORTANT
        if event_time >= datetime(2024, 6, 25, 12, 00):
            global stop_streaming                                  # Variable global stop_streaming
            stop_streaming = True
            if self.position != 0:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL AND STOP") 
                self.position = 0
            else: 
                print("STOP")

        #more stop examples
        # if self.trades >= value
        # if self.cum_profits > value

        # print out
        print("Time: {}| Price: {} ".format(event_time,close))
        #print(".", end = "", flush = True)   # Just print something to get a feedback

        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first,high,low,close,volume,complete]

        # Cuando se complete la barra aplicaremos la estrategia
        if complete == True:
            self.define_strategy()
            self.execute_trades()


    def define_strategy(self):

        df = self.data.copy()

        #************* Definir estrategia aquí *********************
        df = df[["Close","Volume"]].copy()
        df["returns"] = np.log(df["Close"]/df["Close"].shift(1))
        df["Volume_Change"] = np.log(df["Volume"].div(df["Volume"].shift(1)))
        df.loc[df["Volume_Change"] > 3, "Volume_Change"] = np.nan
        df.loc[df["Volume_Change"] < -3, "Volume_Change"] = np.nan

        cond1 = df["returns"] >= self.return_thresh
        cond2 = df["Volume_Change"].between(self.volume_thresh[0], self.volume_thresh[1])

        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #**************************************************************

        self.prepared_data = df.copy()

    def execute_trades(self):
        if self.prepared_data["position"].iloc[-1] == 1:    # Si la posición es larga ---> Mantenemos en largo
            if self.position == 0:                          # Pero si es neutra... creamos una orden de compra de mercado.
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING LONG")
            self.position = 1  # Vuelve a ser posición larga
        elif self.prepared_data["position"].iloc[-1] == 0: # Si la posición es neutral, se mantiene neutral
            if self.position == 1:                          # Pero si es larga... creamos una orden de venta de mercado.
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0  # Vuelve a ser posición neutra

    def report_trade(self, order, going):
        
        # Extract data from order object
        side = order["side"]
        time = pd.to_datetime(order["transactTime"], unit = "ms")
        base_units = float(order["executedQty"])
        quote_units = float(order["cummulativeQuoteQty"])
        price = round(quote_units/ base_units, 5)

        # Calculate trading profits, incrementamos por cada tradeo que hacemos
        self.trades += 1
        if side == "BUY":
            self.trade_values.append(-quote_units)    # Lo que gastamos
        elif side == "SELL":
            self.trade_values.append(quote_units)      # Lo que recibimos

        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]),3)
            cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            cum_profits = round(np.sum(self.trade_values[:-1]), 3)

        # print trade report
        print(2 * "\n" + 100 * "-")
        print("{} | {}".format(time, going))
        print("{} | Base Units = {} | Quote_Units = {} | Price = {}".format(time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(time, real_profit, cum_profits))
        print(100 * "-" + "\n")

In [17]:
symbol = "BTCUSDT"
bar_length = "1m"
return_thresh = 0
volume_thresh = [-3,3]
units = 0.001       
position = 0

In [18]:
trader = LongOnlyTrader(symbol = symbol, bar_length = bar_length, return_thresh = return_thresh,
                        volume_thresh = volume_thresh, units = units, position = position)

In [19]:
stop_streaming = False

In [20]:
async def start_trading(historical_days):
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    
    if trader.bar_length in trader.available_intervals:
        trader.get_most_recent(symbol = trader.symbol, interval = trader.bar_length,
                             days = historical_days)
        ts = bm.kline_socket(symbol = trader.symbol, interval = trader.bar_length)        
    # "else" to be added later in the course
    
    async with ts as tscm:
        while True: # This is just an example to limit the number of messages. Remove or adjust as needed.
            res = await tscm.recv()
            trader.stream_candles(res)
            if stop_streaming:
                break

    await client.close_connection()
await start_trading(historical_days = 1/24)

Time: 2024-06-20 08:46:55.665000| Price: 65650.27 
Time: 2024-06-20 08:46:58.488000| Price: 65647.19 
Time: 2024-06-20 08:47:00.006000| Price: 65647.2 


BinanceAPIException: APIError(code=-2014): API-key format invalid.